<a href="https://colab.research.google.com/github/HectorArambulo/DerrickSherrill.com/blob/master/codigo_proyeccion_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

# Suprimir advertencias de convergencia
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Función para limpiar los datos
def clean_data(df):
    df = df.copy()
    df.columns = df.columns.str.strip()
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
    numeric_cols = df.columns.drop('Fecha')
    for col in numeric_cols:
        df[col] = df[col].astype(str).str.replace(',', '').str.replace(' ', '').astype(float)
    return df

# Función para calcular perfiles típicos
def calculate_typical_profiles(data):
    days_of_week = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    profiles = {}
    for day in range(7):
        day_data = data[data['DayOfWeek'] == day]
        profiles[days_of_week[day]] = day_data.groupby(day_data['Fecha'].dt.hour)['Ejecutado'].mean()
    return profiles

# Función para proyectar la demanda para un día específico
def project_demand(data, typical_profiles, selected_date):
    day_of_week = pd.to_datetime(selected_date).dayofweek
    days_of_week = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    typical_profile = typical_profiles[days_of_week[day_of_week]]

    # Filtrar los datos observados para el día seleccionado
    filtered_data = data[data['Fecha'].dt.date == pd.to_datetime(selected_date).date()]

    # Ajustar la proyección utilizando datos observados
    adjusted_projection = typical_profile.copy()
    for hour, demand in zip(filtered_data['Fecha'].dt.hour, filtered_data['Ejecutado']):
        adjusted_projection.loc[hour] = demand

    # Suavización exponencial para proyectar tendencias
    try:
        model = ExponentialSmoothing(adjusted_projection, trend="add", seasonal=None, initialization_method="estimated")
        hw_fit = model.fit()
        projected_demand = hw_fit.predict(start=0, end=23)
    except Exception as e:
        print("Advertencia: Fallo en la optimización de Holt-Winters. Se utiliza interpolación lineal.")
        projected_demand = adjusted_projection.interpolate(method='linear').fillna(method='bfill').fillna(method='ffill')

    return projected_demand

# Función para completar datos reales con proyección para horas faltantes
def complete_with_projection(data, selected_date, projected_demand):
    filtered_data = data[data['Fecha'].dt.date == pd.to_datetime(selected_date).date()]
    observed_hours = filtered_data['Fecha'].dt.hour.unique()
    missing_hours = set(range(24)) - set(observed_hours)

    if missing_hours:
        missing_demand = {hour: projected_demand[hour] for hour in missing_hours}
        return missing_demand
    return {}

# Función para obtener referencia de una semana anterior
def get_reference_week_data(data, selected_date):
    reference_date = pd.to_datetime(selected_date) - pd.Timedelta(days=7)
    reference_data = data[data['Fecha'].dt.date == reference_date.date()]
    return reference_data, reference_date

# Función para obtener referencia del año anterior considerando el mismo día de la semana y mes
def get_reference_year_data(data, selected_date):
    selected_day_of_week = pd.to_datetime(selected_date).dayofweek
    selected_month = pd.to_datetime(selected_date).month
    year_ago = pd.to_datetime(selected_date) - pd.DateOffset(years=1)

    # Filtrar datos del mismo mes y día de la semana del año anterior
    year_ago_data = data[
        (data['Fecha'].dt.month == selected_month) &
        (data['Fecha'].dt.dayofweek == selected_day_of_week)
    ]
    if not year_ago_data.empty:
        # Seleccionar la fecha más cercana al día consultado
        closest_date = year_ago_data.iloc[(year_ago_data['Fecha'] - year_ago).abs().argsort()].iloc[0]
        return year_ago_data[year_ago_data['Fecha'].dt.date == closest_date['Fecha'].date()], closest_date['Fecha']
    return pd.DataFrame(), None

# Cargar y limpiar los datos
uploaded_file_path = 'HISTORICO_MAXIMA_DEMANDA_LIMP.xlsx'  # Cambia el nombre al archivo subido
historical_data = pd.ExcelFile(uploaded_file_path)
historical_df = historical_data.parse('Hoja1')
historical_df = clean_data(historical_df)

# Agregar columna para el día de la semana
historical_df['DayOfWeek'] = historical_df['Fecha'].dt.dayofweek

# Calcular perfiles típicos
typical_profiles = calculate_typical_profiles(historical_df)

# Función principal
def run_analysis():
    while True:
        try:
            # Solicitar al usuario el día para proyectar
            selected_date = input("Ingrese la fecha para proyectar (YYYY-MM-DD): ")

            # Determinar si la fecha es histórica o futura
            is_future_date = pd.to_datetime(selected_date) > historical_df['Fecha'].max()

            # Crear las etiquetas para el eje X
            hours = [f"{h:02}:00" for h in range(24)]

            # Generar la proyección si es necesario
            projected_demand = project_demand(historical_df, typical_profiles, selected_date)

            # Filtrar datos históricos para el día seleccionado
            filtered_data = historical_df[historical_df['Fecha'].dt.date == pd.to_datetime(selected_date).date()]

            # Completar datos faltantes con proyección
            missing_demand = complete_with_projection(historical_df, selected_date, projected_demand)

            # Obtener datos de referencia de la semana anterior
            week_ago_data, week_ago_date = get_reference_week_data(historical_df, selected_date)

            # Obtener datos de referencia del año anterior
            year_ago_data, year_ago_date = get_reference_year_data(historical_df, selected_date)

            # Crear el gráfico
            fig = go.Figure()

            # Agregar datos históricos disponibles
            if not filtered_data.empty:
                fig.add_trace(go.Scatter(
                    x=filtered_data['Fecha'].dt.strftime('%H:%M'),
                    y=filtered_data['Ejecutado'],
                    mode='lines+markers',
                    name='Demanda Real',
                    line=dict(shape='spline', color='red')
                ))

            # Agregar datos de referencia de la semana anterior
            if not week_ago_data.empty:
                fig.add_trace(go.Scatter(
                    x=week_ago_data['Fecha'].dt.strftime('%H:%M'),
                    y=week_ago_data['Ejecutado'],
                    mode='lines',
                    name=f'Demanda Semana Anterior ({week_ago_date.strftime("%Y-%m-%d")})',
                    line=dict(shape='spline', color='purple')
                ))

            # Agregar datos de referencia del año anterior
            if not year_ago_data.empty:
                fig.add_trace(go.Scatter(
                    x=year_ago_data['Fecha'].dt.strftime('%H:%M'),
                    y=year_ago_data['Ejecutado'],
                    mode='lines',
                    name=f'Demanda Año Anterior ({year_ago_date.strftime("%Y-%m-%d")})',
                    line=dict(shape='spline', color='brown')
                ))

            # Agregar datos proyectados para horas faltantes
            if missing_demand:
                fig.add_trace(go.Scatter(
                    x=[f"{h:02}:00" for h in missing_demand.keys()],
                    y=list(missing_demand.values()),
                    mode='lines',
                    name='Proyección de Horas Faltantes',
                    line=dict(shape='spline', dash='dot', color='blue')
                ))

            # Configurar diseño del gráfico
            fig.update_layout(
                title=f'Proyección de Demanda para el {selected_date}',
                xaxis_title='Hora',
                yaxis_title='Demanda (MW)',
                yaxis=dict(tickformat=',d'),
                xaxis=dict(tickmode='array', tickvals=hours),
                template='plotly_white'
            )

            # Mostrar el gráfico
            fig.show()

        except Exception as e:
            print(f"Ocurrió un error: {e}")

        # Preguntar si se desea consultar otra fecha
        repeat = input("¿Desea consultar otra fecha? (s/n): ").strip().lower()
        if repeat != 's':
            print("Finalizando consulta.")
            break

# Ejecutar el análisis
run_analysis()
